Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [ ]:
!pip freeze | grep superlinked

In [1]:
from pathlib import Path
import sys

# depending on the user's setup
# we will try to find the superlinked_app directory
# and add it to the sys.path

cwd = Path.cwd()
if cwd.name == "superlinked-recipes":
    project_dir = cwd / "projects" / "recipe-search"
elif cwd.name == "notebooks":
    project_dir = cwd.parent
else:
    project_dir = cwd

superlinked_app_dir = project_dir / "superlinked_app"
assert superlinked_app_dir.exists(), (
    f"{superlinked_app_dir} does not exist\n"
    "are you sure you are in the recipe-search/notebooks directory?"
)

if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))
    print("project_dir is added to sys.path")
else:
    print("project_dir is already in sys.path")

project_dir is added to sys.path


### This is to use the collection_name=sherlockrecipes

In [2]:
import os
os.environ["APP_ID"] = "sherlockrecipes"
os.environ["SUPERLINKED_NLQ_ENABLE_VALIDATION_FOLLOWUP"] = "false"


In [3]:
from superlinked import framework as sl

from superlinked_app.index import index, recipe_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

import pandas as pd

/home/biso/development/my_projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


17:56:26 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/paraphrase-MiniLM-L3-v2


Batches: 100%|██████████| 1/1 [00:00<00:00, 100.14it/s]

17:56:28 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
17:56:28 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
17:56:28 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
17:56:28 superlinked.framework.dsl.index.index INFO   initialized index


In [4]:
source = sl.InteractiveSource(recipe_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[index],
    vector_database=vector_database,
)
app = executor.run()

17:56:29 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
17:56:29 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/exists "HTTP/1.1 200 OK"
17:56:29 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes "HTTP/1.1 200 OK"
17:56:29 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/index?wait=true "HTTP/1.1 200 OK"
17:56:30 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/index?wait=true "HTTP/1.1 200 OK"
17:56:30 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes

In [5]:
params = {
    "natural_query": "quick dessert with chocolate from French cuisine",
    "limit": 3,
}

result = app.query(query, **params)

# sl.PandasConverter.to_pandas(result)
# Print search parameters and convert results to pandas DataFrame
# Show all rows

print(result.metadata.search_params)
df = sl.PandasConverter.to_pandas(result)
df

17:56:38 httpx INFO   HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.15it/s]

17:56:42 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


17:56:42 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/points/query "HTTP/1.1 200 OK"
17:56:42 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'name_query': 'quick dessert', 'similar_name_weight': 1.0, 'ingredients_query': 'chocolate', 'similar_ingredients_weight': 1.0, 'instructions_query': 'quick', 'similar_instructions_weight': 1.0, 'limit': 3, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'min_rating': None, 'max_rating': None, 'min_prep_time': None, 'max_prep_time': 30.0, 'min_cook_time': None, 'max_cook_time': 30.0, 'max_calories': None, 'categories_include_all': None, 'categories_include_any': ['Dess

,Name,Ingredient_Names_Text,Instructions,Rating_Value,Preparation_Time,Cooking_Time,Category,Cuisine,Calories,Carbohydrates,...,Saturated_Fat,Sodium,Sugar,Fat,Unsaturated_Fat,Nutrition,Ingredients,URL,id,similarity_score
0,Creamy Chocolate Mousse Pie,miniature_marshmallows milk_chocolate_candy mi...,"Heat marshmallows, bar chocolate, and milk in ...",4.8,20.0,10.0,[Dessert],[French],399.0,29.0,...,17.0,142.0,17.0,31.0,0.0,"{""Calories"": ""399 kcal"", ""Carbohydrates"": ""29 ...","[{'ingredient': 'miniature marshmallows', 'qua...",https://www.allrecipes.com/recipe/229164/cream...,rec122,-0.123579


In [6]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,name,ingredents,instructions,rating,prep_time,cooking_time,calories,protein
0,rec122,0.119303,0.123086,0.032608,0.0,-0.198874,-0.199702,0.0,0.0


In [7]:
params = {
    "natural_query": "quick vegan dinner less than 370 calories from Indian cuisine",
    "limit": 3,
}

result = app.query(query, **params)

# sl.PandasConverter.to_pandas(result)
# Print search parameters and convert results to pandas DataFrame
# Show all rows

print(result.metadata.search_params)
df = sl.PandasConverter.to_pandas(result)
df

17:56:46 httpx INFO   HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.41it/s]

17:56:50 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


17:56:51 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/points/query "HTTP/1.1 200 OK"
17:56:51 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'name_query': 'quick vegan dinner', 'similar_name_weight': 1.0, 'ingredients_query': 'vegan', 'similar_ingredients_weight': 1.0, 'instructions_query': 'quick', 'similar_instructions_weight': 1.0, 'limit': 3, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'min_rating': None, 'max_rating': None, 'min_prep_time': None, 'max_prep_time': 30.0, 'min_cook_time': None, 'max_cook_time': 30.0, 'max_calories': 370.0, 'categories_include_all': None, 'categories_include_any': ['Di

""


In [8]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

""


In [ ]:
params = {
    "natural_query": "quick dinner with onion less than 3070 calories and cooking time less than 40 minutes",
    "limit": 3,
}

result = app.query(query, **params)

# sl.PandasConverter.to_pandas(result)
# Print search parameters and convert results to pandas DataFrame
# Show all rows

print(result.metadata.search_params)
df = sl.PandasConverter.to_pandas(result)
df

In [ ]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

### The following examples are with manual parameters

In [12]:
# Example manual query parameters 
params = {
    "name_query": "",              # Text to search in recipe names
    "ingredients_query": "cream", # Text to search in ingredients
    "instructions_query": "",      # Text to search in instructions
    "limit": 5,                                  # Number of results to return

    # Weights for similarity spaces (adjust as needed)
    "name_weight": 1.0,
    "ingredients_weight": 1.0,
    "instructions_weight": 0.0,

    # Similarity weights (can be same as above or different)
    "similar_name_weight": 1.0,
    "similar_ingredients_weight": 1.0,
    "similar_instructions_weight": 0.0,

    # Numerical filters (set to None or appropriate values)
    "min_rating": 0.0,
    #"max_rating": 15.0,
    "min_prep_time": 0.0,
    "max_prep_time": 40.0,
    "min_cook_time": 0.0,
    "max_cook_time": 40.0,
    #"max_calories": 400.0,

    # Categorical filters (empty list or specify categories/cuisines)
    # Remove or set to None to disable categories filtering
    "categories_include_any": ["Dessert"],
    #"categories_exclude": [],
    # Remove or set to None to disable cuisine filtering
    "cuisines_include_any": ["French"],
    # "cuisines_exclude": [],
}

# Run the query with manual parameters
result = app.query(query, **params)

# Print search parameters and convert results to pandas DataFrame
# Show all rows

print(result.metadata.search_params)
df = sl.PandasConverter.to_pandas(result)
df


Batches: 100%|██████████| 1/1 [00:00<00:00, 33.09it/s]

18:00:07 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


18:00:07 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/points/query "HTTP/1.1 200 OK"
18:00:07 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'name_query': '', 'similar_name_weight': 1.0, 'ingredients_query': 'cream', 'similar_ingredients_weight': 1.0, 'instructions_query': '', 'similar_instructions_weight': 0.0, 'limit': 5, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'min_rating': 0.0, 'max_rating': None, 'min_prep_time': 0.0, 'max_prep_time': 40.0, 'min_cook_time': 0.0, 'max_cook_time': 40.0, 'max_calories': None, 'categories_include_all': None, 'categories_include_any': ['Dessert'], 'categories_exclud

,Name,Ingredient_Names_Text,Instructions,Rating_Value,Preparation_Time,Cooking_Time,Category,Cuisine,Calories,Carbohydrates,...,Saturated_Fat,Sodium,Sugar,Fat,Unsaturated_Fat,Nutrition,Ingredients,URL,id,similarity_score
0,Creamy Chocolate Mousse Pie,miniature_marshmallows milk_chocolate_candy mi...,"Heat marshmallows, bar chocolate, and milk in ...",4.8,20.0,10.0,[Dessert],[French],399.0,29.0,...,17.0,142.0,17.0,31.0,0.0,"{""Calories"": ""399 kcal"", ""Carbohydrates"": ""29 ...","[{'ingredient': 'miniature marshmallows', 'qua...",https://www.allrecipes.com/recipe/229164/cream...,rec122,0.355058


In [13]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,name,ingredents,instructions,rating,prep_time,cooking_time,calories,protein
0,rec122,0.051456,0.303602,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Example manual query parameters
params = {
    "name_query": "Pizza",              # Text to search in recipe names
    "ingredients_query": "greek_yogurt", # Text to search in ingredients
    "instructions_query": "Preheat",      # Text to search in instructions
    "limit": 5,                                  # Number of results to return

    # Weights for similarity spaces (adjust as needed)
    "name_weight": 1.0,
    "ingredients_weight": 1.0,
    "instructions_weight": 1.0,

    # Similarity weights (can be same as above or different)
    "similar_name_weight": 1.0,
    "similar_ingredients_weight": 1.0,
    "similar_instructions_weight": 1.0,

    # Numerical filters (set to None or appropriate values)
    "min_rating": 4.0,
    #"max_rating": 15.0,
    "min_prep_time": 0.0,
    "max_prep_time": 9000.0,
    "min_cook_time": 0.0,
    "max_cook_time": 1000.0,
    "max_calories": 400.0,

    # Categorical filters (empty list or specify categories/cuisines)
    # Remove or set to None to disable categories filtering
    #"categories_include_any": ["Dinner"],
    #"categories_exclude": [],
    # Remove or set to None to disable cuisine filtering
    "cuisines_include_any": ["Italian"],
    # "cuisines_exclude": [],
}

# Run the query with manual parameters
result = app.query(query, **params)

# Print search parameters and convert results to pandas DataFrame
# Show all rows

print(result.metadata.search_params)
df = sl.PandasConverter.to_pandas(result)
df


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.64it/s]

18:00:21 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


18:00:21 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/points/query "HTTP/1.1 200 OK"
18:00:21 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'name_query': 'Pizza', 'similar_name_weight': 1.0, 'ingredients_query': 'greek_yogurt', 'similar_ingredients_weight': 1.0, 'instructions_query': 'Preheat', 'similar_instructions_weight': 1.0, 'limit': 5, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'min_rating': 4.0, 'max_rating': None, 'min_prep_time': 0.0, 'max_prep_time': 9000.0, 'min_cook_time': 0.0, 'max_cook_time': 1000.0, 'max_calories': 400.0, 'categories_include_all': None, 'categories_include_any': None, '

,Name,Ingredient_Names_Text,Instructions,Rating_Value,Preparation_Time,Cooking_Time,Category,Cuisine,Calories,Carbohydrates,...,Saturated_Fat,Sodium,Sugar,Fat,Unsaturated_Fat,Nutrition,Ingredients,URL,id,similarity_score
0,Two-Ingredient Pizza Dough,self-rising_flour greek_yogurt spray,Gather all ingredients. Preheat the oven to 50...,4.6,15.0,10.0,[Dinner],[Italian],116.0,18.0,...,1.0,314.0,1.0,3.0,0.0,"{""Calories"": ""116 kcal"", ""Carbohydrates"": ""18 ...","[{'ingredient': 'self-rising flour', 'quantity...",https://www.allrecipes.com/recipe/244447/two-i...,rec136,0.551015


In [15]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,name,ingredents,instructions,rating,prep_time,cooking_time,calories,protein
0,rec136,0.22264,0.209723,0.118651,0.0,0.0,0.0,0.0,0.0


In [16]:
# Example manual query parameters 
params = {
    "name_query": "pasta",              # Text to search in recipe names
    "ingredients_query": "pasta", # Text to search in ingredients
    "instructions_query": "",      # Text to search in instructions
    "limit": 5,                                  # Number of results to return

    # Weights for similarity spaces (adjust as needed)
    "name_weight": 1.0,
    "ingredients_weight": 1.0,
    "instructions_weight": 0.0,

    # Similarity weights (can be same as above or different)
    "similar_name_weight": 1.0,
    "similar_ingredients_weight": 1.0,
    "similar_instructions_weight": 0.0,

    # Numerical filters (set to None or appropriate values)
    "min_rating": 0.0,
    #"max_rating": 15.0,
    "min_prep_time": 0.0,
    "max_prep_time": 40.0,
    "min_cook_time": 0.0,
    "max_cook_time": 40.0,
    "max_calories": 500.0,

    # Categorical filters (empty list or specify categories/cuisines)
    # Remove or set to None to disable categories filtering
    "categories_include_any": ["Lunch", "Dinner"],
    #"categories_exclude": [],
    # Remove or set to None to disable cuisine filtering
    "cuisines_include_any": ["Italian"],
    # "cuisines_exclude": [],
}

# Run the query with manual parameters
result = app.query(query, **params)

# Print search parameters and convert results to pandas DataFrame
# Show all rows

print(result.metadata.search_params)
df = sl.PandasConverter.to_pandas(result)
df


Batches: 100%|██████████| 1/1 [00:00<00:00, 75.45it/s]

18:00:21 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
18:00:21 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/sherlockrecipes/points/query "HTTP/1.1 200 OK"
18:00:21 superlinked.framework.dsl.executor.query.query_executor INFO   executed query


{'name_query': 'pasta', 'similar_name_weight': 1.0, 'ingredients_query': 'pasta', 'similar_ingredients_weight': 1.0, 'instructions_query': '', 'similar_instructions_weight': 0.0, 'limit': 5, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'min_rating': 0.0, 'max_rating': None, 'min_prep_time': 0.0, 'max_prep_time': 40.0, 'min_cook_time': 0.0, 'max_cook_time': 40.0, 'max_calories': 500.0, 'categories_include_all': None, 'categories_include_any': ['Lunch', 'Dinner'], 'categories_exclude': None, 'cuisines_include_all': None, 'cuisines_include_any': ['Italian'], 'cuisines_exclude': None, 'natural_query': None, 'system_prompt_param__': "Extract search parameters from the user's natural language query about recipes.\nGuidelines:\n- Identif

,Name,Ingredient_Names_Text,Instructions,Rating_Value,Preparation_Time,Cooking_Time,Category,Cuisine,Calories,Carbohydrates,...,Saturated_Fat,Sodium,Sugar,Fat,Unsaturated_Fat,Nutrition,Ingredients,URL,id,similarity_score
0,Two-Ingredient Pizza Dough,self-rising_flour greek_yogurt spray,Gather all ingredients. Preheat the oven to 50...,4.6,15.0,10.0,[Dinner],[Italian],116.0,18.0,...,1.0,314.0,1.0,3.0,0.0,"{""Calories"": ""116 kcal"", ""Carbohydrates"": ""18 ...","[{'ingredient': 'self-rising flour', 'quantity...",https://www.allrecipes.com/recipe/244447/two-i...,rec136,0.309097
1,Trofie alla Genovese,pine_nuts basil parmesan_cheese extra-virgin_o...,Start by making the pesto: Heat a dry pan over...,0.0,20.0,40.0,[Dinner],[Italian],470.0,26.0,...,7.0,437.0,2.0,38.0,0.0,"{""Calories"": ""470 kcal"", ""Carbohydrates"": ""26 ...","[{'ingredient': 'pine nuts', 'quantity': '0.33...",https://www.allrecipes.com/recipe/284486/trofi...,rec217,0.271705


In [17]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,name,ingredents,instructions,rating,prep_time,cooking_time,calories,protein
0,rec136,0.164990,0.144107,0.0,0.0,0.0,0.0,0.0,0.0
1,rec217,0.071625,0.200080,0.0,0.0,0.0,0.0,0.0,0.0
